In [1]:
! pip install pycld3 stanza -q

In [2]:
import pandas as pd
import cld3
import numpy as np

In [3]:
df = pd.read_csv("/kaggle/input/nlp-telegram-locations-extractions/test.csv")

In [4]:
df.head()

,text_id,text
0,0,#нампишут на Троещине маньяк забрал жизни уже ...
1,1,"Патрульні взяли участь у ""Забігу в вишиванках ..."
2,2,"«Лети, Соколе…»: у Києві попрощались із загибл..."
3,3,Ваш вибір?\n\nhttps://vm.tiktok.com/ZMj8etTtY
4,4,Кабмін звільнив голову Держагентства з управлі...


In [5]:
def set_label_cld3(message: str) -> str:
  label = cld3.get_language(message)
  if label[0] == 'uk':
    return 'u'
  else:
    return 'r'

In [6]:
df['lang'] =  np.vectorize(set_label_cld3)(df['text'])

In [7]:
df.head()

,text_id,text,lang
0,0,#нампишут на Троещине маньяк забрал жизни уже ...,r
1,1,"Патрульні взяли участь у ""Забігу в вишиванках ...",u
2,2,"«Лети, Соколе…»: у Києві попрощались із загибл...",u
3,3,Ваш вибір?\n\nhttps://vm.tiktok.com/ZMj8etTtY,u
4,4,Кабмін звільнив голову Держагентства з управлі...,u


In [8]:
df["lang"].value_counts()

lang
u    427
r     50
Name: count, dtype: int64

In [9]:
df_ua = df[df["lang"] == "u"].reset_index(drop=True)
df_ru = df[df["lang"] == "r"].reset_index(drop=True)

In [10]:
df_ru.head()

,text_id,text,lang
0,0,#нампишут на Троещине маньяк забрал жизни уже ...,r
1,10,"Загроза для Житомирщини, Київщини.",r
2,12,Найдена собака <корге> забежала в маршрутку на...,r
3,22,Ще ракета на Київ.,r
4,30,Национальный рекорд: в Дарницком районе сплели...,r


In [11]:
! python -m spacy download ru_core_news_sm -q
! python -m spacy download uk_core_news_sm -q

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_sm')


In [12]:
import spacy
import stanza
nlp_ru = spacy.load('ru_core_news_sm')
# nlp_ua = spacy.load('uk_core_news_sm')

nlp_ua = stanza.Pipeline(lang='uk', processors='tokenize,ner')
# doc=nlp_ua('Патрульні взяли участь у Забігу в вишиванках у Києві після чого побігли у Львів')
# print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

In [13]:
df.head()

,text_id,text,lang
0,0,#нампишут на Троещине маньяк забрал жизни уже ...,r
1,1,"Патрульні взяли участь у ""Забігу в вишиванках ...",u
2,2,"«Лети, Соколе…»: у Києві попрощались із загибл...",u
3,3,Ваш вибір?\n\nhttps://vm.tiktok.com/ZMj8etTtY,u
4,4,Кабмін звільнив голову Держагентства з управлі...,u


In [14]:
locations = []
markers = []
for index, row in df.iterrows():
    m = []
    l = []
    if row["lang"] == "u":
        doc = nlp_ua(row["text"])
        for ent in doc.ents:
            if ent.type == "LOC":  # Filter for geographical and location entities
                m.append((ent.start_char, ent.end_char))
                l.append(str(ent.text))
    else:
        doc = nlp_ru(row["text"])
        for ent in doc.ents:
            if ent.label_ == "LOC":  # Filter for geographical and location entities
                m.append((ent.start_char, ent.end_char))
                l.append(str(ent))
    locations.append(str(l))
    markers.append(str(m))

In [15]:
df["locations"] = locations

In [16]:
df["loc_markers"] = markers

In [17]:
df.drop(columns=["lang"], inplace=True)
df.set_index('text_id', inplace=True)

In [18]:
df

,text,locations,loc_markers
text_id,,,
0,#нампишут на Троещине маньяк забрал жизни уже ...,['Троещине'],"[(13, 21)]"
1,"Патрульні взяли участь у ""Забігу в вишиванках ...",[],[]
2,"«Лети, Соколе…»: у Києві попрощались із загибл...","['Києві', 'Донбасі', 'Курдюмівка', 'Київраді',...","[(19, 24), (172, 179), (193, 203), (330, 338),..."
3,Ваш вибір?\n\nhttps://vm.tiktok.com/ZMj8etTtY,[],[]
4,Кабмін звільнив голову Держагентства з управлі...,[],[]
...,...,...,...
472,Більше машинок місту машинок! 😆\n\n(с) Олексан...,[],[]
473,В Охматдиті почали будівництво першого в Украї...,['Україні'],"[(41, 48)]"
474,На території столиці введений тепловий режим ☀...,['Київ'],"[(216, 220)]"


In [19]:
df.to_csv('/kaggle/working/submission.csv')

In [20]:
dff = pd.read_csv("/kaggle/input/nlp-telegram-locations-extractions/labeling_sample.csv")

In [21]:
dff.head()

,text,locations,loc_markers
0,Ще маємо всього 3 такі сувенірні ящички від БК...,[],[]
1,В результате массированной атаки врага ночью в...,"['Фастовском', 'Бучанском']","[(47, 57), (60, 69)]"
2,Дивно що на сам спортивний майданчик не заїхав...,"[""вул. Солом'янська, 28""]","[(61, 82)]"
3,Найяскравіший і найбільший Місяць 2023 року,[],[]
4,😱 Жесть на Троєщині хлопця пирнули ножем на зу...,['Троєщині'],"[(11, 19)]"
